In [1]:
import pandas as pd
import numpy as np
import torch
import torch.utils.data
import torch.nn as nn
from torch.autograd import Variable
from sklearn.model_selection import train_test_split
import joblib
from tqdm import tqdm
import matplotlib.pyplot as plt
from copy import deepcopy

In [2]:
from isaac.utils import get_cuda_device_if_available, create_directory
device = get_cuda_device_if_available()
print(device)

data_directory = "r_theta/"
create_directory(data_directory)
create_directory("models/r_theta/")
create_directory("scalers")

cuda:0


In [3]:
from isaac.dataset import read_dataset, prepare_dataset
from isaac.utils import plot_confusion_matrix
from isaac.constants import BASIC_TRAINING_COLS, FORCE_CLASS_COLS, MASS_CLASS_COLS
from isaac.training import training_loop
from isaac.models import MultiBranchModel, initialise_model
from isaac.evaluation import get_best_model_and_its_accuracy

In [4]:
BATCH_SIZE = 128
EPOCHS = 100
NORMALISE_DATA = True
STEP_SIZE = 3
SEQ_END = 2700

INPUT_DIM = len(BASIC_TRAINING_COLS)    # input dimension
HIDDEN_DIM = 25  # hidden layer dimension
N_LAYERS = 4     # number of hidden layers
OUTPUT_DIM = 3   # output dimension
DROPOUT = 0.5

network_params = (INPUT_DIM, HIDDEN_DIM, OUTPUT_DIM, DROPOUT)

## Read dataset and preprocess it

In [5]:
train_trials = read_dataset("data/train_passive_trials.h5", cols=BASIC_TRAINING_COLS)
val_trials = read_dataset("data/val_passive_trials.h5", cols=BASIC_TRAINING_COLS)

100%|██████████| 1000/1000 [00:08<00:00, 114.98it/s]


In [6]:
RTHETA_ATTRS = [obj+"."+attr for obj in ["o1", "o2", "o3", "o4"] for attr in ["x", "y", "r", "theta"]]

def add_r_theta_attributes(trials):
    for trial in tqdm(trials):
        for obj in ["o1", "o2", "o3", "o4"]:
            trial[obj+".r"] = (trial[obj+".vx"]**2 + trial[obj+".vy"]**2)**0.5
            trial[obj+".theta"] = (np.arctan2(trial[obj+".vx"], trial[obj+".vy"]) * 180 / np.pi)

add_r_theta_attributes(train_trials)
add_r_theta_attributes(val_trials)

100%|██████████| 1000/1000 [01:11<00:00, 13.99it/s]


In [7]:
train_trials[0][RTHETA_ATTRS].head()

,o1.x,o1.y,o1.r,o1.theta,o2.x,o2.y,o2.r,o2.theta,o3.x,o3.y,o3.r,o3.theta,o4.x,o4.y,o4.r,o4.theta
0,1.31253,3.013948,5.507783,132.610561,4.258775,2.160018,12.634763,-43.695787,5.422599,2.014424,8.036512,-75.857927,4.985966,2.753544,11.011006,121.950045
1,1.38005,2.951850,5.504239,132.603147,4.113570,2.312260,12.623376,-43.645417,5.292690,2.047320,8.040282,-75.790595,5.141500,2.656210,11.008778,122.040371
2,1.44754,2.889800,5.500725,132.595496,3.968610,2.464420,12.609304,-43.611090,5.162780,2.080590,8.046592,-75.634300,5.296840,2.558500,11.011026,122.167324
3,1.51500,2.827800,5.497245,132.587777,3.823870,2.616460,12.595276,-43.591654,5.022060,2.065790,8.489894,-96.006138,5.462840,2.508930,10.394279,106.626837
4,1.58242,2.765840,5.493810,132.579773,3.679300,2.768370,12.582110,-43.580452,4.881720,2.051290,8.464804,-95.895654,5.473250,2.467860,9.910181,-100.935681


## Train model and plot loss and accuracy

In [8]:
N_MODELS = 3

stats_dfs = []
loaders, scaler = prepare_dataset([train_trials, val_trials], 
                                  class_columns=[list(MASS_CLASS_COLS), list(FORCE_CLASS_COLS)], 
                                  multiclass=True,
                                  batch_size=BATCH_SIZE, normalise_data=NORMALISE_DATA,
                                  device=device, training_columns=RTHETA_ATTRS)


for seed in range(N_MODELS):
    df = pd.DataFrame()

    model, error, optimizer = initialise_model(network_params, lr=0.01, seed=seed, device=device, arch=MultiBranchModel)
    epoch_losses, epoch_accuracies, [best_mass_model, best_force_model] = training_loop(model, optimizer, 
                                                                                        error,
                                                                                        loaders[0], loaders[1], 
                                                                                        EPOCHS, seq_end=SEQ_END,
                                                                                        step_size=STEP_SIZE,
                                                                                        multibranch=True)

    torch.save(best_mass_model.state_dict(), "models/r_theta/best_mass_model_seed_%d.pt" % seed)
    torch.save(best_force_model.state_dict(), "models/r_theta/best_force_model_seed_%d.pt" % seed)
    
    
    train_accuracies = np.array(epoch_accuracies[0])
    val_accuracies = np.array(epoch_accuracies[1]) 
    
    df["Epoch"] = np.arange(EPOCHS)
    df["Mass Loss"] = epoch_losses[:, 0]
    df["Force Loss"] = epoch_losses[:, 1]
    df["Mass Train Accuracy"] = train_accuracies[:, 0]
    df["Mass Val Accuracy"] = val_accuracies[:, 0]
    df["Force Train Accuracy"] = train_accuracies[:, 1]
    df["Force Val Accuracy"] = val_accuracies[:,1]
    df["seed"] = str(seed)
    stats_dfs.append(df)
        
stats = pd.concat(stats_dfs)
stats.to_hdf(data_directory+"stats.h5", key="stats")

100%|██████████| 1000/1000 [00:01<00:00, 542.07it/s]
Train_loss: ([0.53783192 0.59829779]) Train_acc: ([76.22857143 73.34285714]) Val_acc: ([64.4 63.4]): 100%|██████████| 100/100 [20:01<00:00, 11.76s/it]
Train_loss: ([0.54278776 0.64404561]) Train_acc: ([74.         64.25714286]) Val_acc: ([61.4 61.3]): 100%|██████████| 100/100 [19:21<00:00, 11.45s/it]
Train_loss: ([0.5234014  0.50845636]) Train_acc: ([76.62857143 76.57142857]) Val_acc: ([66.4 66. ]): 100%|██████████| 100/100 [19:11<00:00, 11.63s/it]


## Save model and scaler

In [9]:
joblib.dump(scaler, "scalers/passive_rtheta_scaler.sk")

['scalers/passive_rtheta_scaler.sk']